# Trabalho 1 Inf450 2023

**Objetivo**

Construir simuladores de arquiteturas com múltiplas unidades funcionais com visualização da execiução.

In [ ]:
import pandas as pd

## Definição do Tempo

Neste exemplo, um simples dicionário é usado para inicializar o tempo de cada unidade. Aqui só tem 2 tipos de instruções.

In [ ]:
Tempo = {}
Tempo["addf"] = 3
Tempo["mulf"] = 5
Tempo["addi"] = 1
Tempo["sd"] = 2
Tempo["ld"] = 2

## Definição do Código de Teste

Para o código de teste ter uma interface bem simples, foi criada uma função de impressão (que pode ser melhorada) e um programa é uma lista de instruções.
Um instrução também está codificada como uma lista. Portanto, o programa é uma lista de listas.

Cada instrução tem seu nome e seus parametros. A versõa inicial, os dois tipos tem 3 parametros, mas para o trabalho, deve ser generalizado pois tem instruções com 1 ou 2 parametros apenas.



In [ ]:
def inst2text(x):
  if(x[0] == 'addi'):
    return x[0]+ " r"+str(x[1])+ ", r"+str(x[2])+ ", r"+str(x[3])
  elif(x[0] == 'sd' or x[0] == 'ld'):
    return x[0]+ " f"+str(x[1])+ ", " + str(x[2])+ "(r"+str(x[3]) + ")"
  else:
    return x[0]+ " f"+str(x[1])+ ", f"+str(x[2])+ ", f"+str(x[3])

### Alguns exemplos de código

In [ ]:
Codigo = [["addf",3,4,5],["mulf",4,3,5]]

In [ ]:
#codigo teste
Codigo = [["addf",3,5,2],["sd",1,0,1]]

In [ ]:
# #codigo teste
Codigo = [["addf",1,2,3],["mulf",3,1,3],["addi",2,2,1],["ld",5,0,2],["addf",4,3,5],["mulf",3,3,5]]

In [ ]:
Codigo = [["addf",3,4,5],["mulf",4,3,5],["addf",3,4,5],["mulf",4,3,5]]

In [ ]:
Codigo = [["addf",3,4,5],["addf",3,4,5],["addf",3,4,5],["addf",3,4,5]]

In [ ]:
Codigo = [["addf",3,4,5],["mulf",3,4,5],["mulf",3,4,5],["addf",3,4,5]]

In [ ]:
Codigo = [["addf",3,4,5],["addf",3,4,5],["addf",3,3,5],["addf",3,4,5]]

In [ ]:
Codigo = [["addf",3,4,5],["addf",4,3,5],["addf",5,3,4],["addf",3,4,5]]

In [ ]:
Codigo = [["addf",3,3,3],["addf",3,3,3],["addf",3,3,3],["addf",3,3,3]]

# Trabalho 1

Nomes e Matriculas dos integrantes

1. Luísa de Souza Ferreira (102026)

2. Lademir Junior (102018)

3. Jonas Magalhães (102010)

4. Felipe Ferreira (102017)

5. Bruno Alencar (102008)

## Parte I

* Adicionar instruções de Ld e Sd com 2 ciclos.
Observe que os campos tem significado diferente já que ld $R_d$, Im ($R_{s1}$) e sd $R_{s2}$, Im ($R_{s1}$), portanto o tratamento tem que ficar atento a posição e significado dos parametros.

```
["ld",3,5,0] # LD x3,0(x5)
["sd",3,5,8] # SD x3,8(x5)
```


* Adicionar a instrução **addi** para inteiros com 1 ciclo apenas, então não terá problemas de dependência de dados. Usar os registrados $R$ para inteiros e $F$ para ponto flutuante.

* Adicionar a parada por conflito no Write-Back.


In [ ]:
from IPython.lib.pretty import Breakable
#@title Parte I


pc = 0
max_ciclos = 18
last = 0
instrucoes = len(Codigo)
L = []
#constroi a tabela
for i in Codigo:
  L.append(inst2text(i))
tabela = { 'prog': L}

L = []
for i in Codigo:
  L.append(" ")

for i in range(max_ciclos):
   tabela[str(i+1)] = L

df = pd.DataFrame(data=tabela)
#

NOP = ["nop",0,0,0]
ciclo = 1
stall = False
Fetch = [NOP,-1]
Decode = [NOP,-1]
WBack = [NOP,-1]
addf = []
addfBAR = []
register = 0
for i in range(Tempo["addf"]):
  addf.append([NOP,-1])
  addfBAR.append([NOP,-1])
mulf = []
mulfBAR = []
for i in range(Tempo["mulf"]):
  mulf.append([NOP,-1])
  mulfBAR.append([NOP,-1])
#acho q n precisa do addi pq é só um ciclo
#iniciando ld/sd(precisa de um para representar os dois, já que eles seguem o "mesmo" fluxo)
ld = []
ldBAR = []
for i in range(Tempo["ld"]):
  ld.append([NOP,-1])
  ldBAR.append([NOP,-1])

addi = [[[NOP,-1]]]
#

#diferenciar por dois motivos: addi grava em outro registrador e também no caso de conflito no W se o ld for com r1 não necessita ter conflito.
writeback_f = {}
writeback_r = {}
#writeback_f_f => para registradores F(tipo flutuante)
#writeback_f_s => para registradores R(tipo inteiro)

forward = 0
n_stall = 0
forward2 = 0 #para o conflito no estagio W
while ciclo < max_ciclos:
  if (not stall):
    FetchBAR = Fetch       # move a instrucao ciclo anterior F -> BARRA
    if (pc < instrucoes):    # Fetch, verifica se ainda tem instrucoes
      Fetch = [Codigo[pc],pc]         # Busca nova instrucao e sua linha na tabela
      df.iloc[pc][str(ciclo)] = "F"  # Marca F na tabela
      pc += 1                         # prepara para buscar a proxima
    else:
      Fetch = [NOP,-1]    # busca NOP

    DecodeBAR = Decode  # move a instrucao ciclo anterior D -> BARRA
    Decode = FetchBAR   # busca a instrucao ciclo anterior F->D
    if ( Decode[1] >= 0):       # senao é NOP marca D na tabela
      df.iloc[Decode[1]][str(ciclo)] = "D"

    #RESOLVENDO POSSÍVEIS CONFLITOS NO W
    if(not Decode[0][0] == "nop" and not Decode[0][0] == "sd"):
      tempo = ciclo + Tempo[Decode[0][0]] + 1
      # print(Decode, tempo)
      for key in writeback_f.keys():
        if writeback_f[key] == tempo: #se tiver algum registrador F que irá escrever no mesmo ciclo, dá um stall
          stall = True
          forward2 = tempo + 1
          break
    #RESOLVENDO DEPENDÊNCIAS DE DADOS
    if Decode[0][2] in writeback_f or Decode[0][3] in writeback_f:# verifica se tem RAW rs1 e rs2
      if(Decode[0][2] in writeback_f and Decode[0][3] in writeback_f): #resolver conflitos desse tipo => [["mulf",3,1,3],["addi",2,2,1],["ld",5,0,2],["addf",4,3,5]], quando vai testar com o útlimo addf
      #tanto f3 quanto f5 estão no writeback_f, então por ordem ele acaba pegando o f5(segundo if) e dava errado :( #https://excalidraw.com/#json=YBtQq_m-daQlj3r1yVlLG,M6HFuD-r6WQO1yYJA8SWpQ
        if(writeback_f[Decode[0][2]] > writeback_f[Decode[0][3]]):
          forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
          register = Decode[0][2]
        else:
          forward = writeback_f[Decode[0][3]] - 1
          register = Decode[0][3]
        stall = True
      elif ( Decode[0][2] in writeback_f):
        forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
        register = Decode[0][2]
        stall = True             # guarda registro de escrita (para )
      elif ( Decode[0][3] in writeback_f):
        forward = writeback_f[Decode[0][3]] - 1
        register = Decode[0][3]
        stall = True                           # marca STALL

    else:                                    # senao tem dependencia mais, registra onde ira escrever
      if ( Decode[0][0] == "addf"):
        writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1
      if ( Decode[0][0] == "mulf"):
        writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1
      #adicionar addi e ld/sd
      if ( Decode[0][0] == "addi"):
        writeback_r[Decode[0][1]] = ciclo+Tempo["addi"]+1
      if ( Decode[0][0] == "ld"): #sd grava na memoria e não em registrador
        writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1

  else: #stall
    n_stall+=1
    DecodeBAR = [NOP,-1]  # Ira propagar NOP para exec
    if ( ciclo == forward2 ):  # se RAW resolveu, libera stall,registro no proximo ciclo
      stall = False
      continue
    if ( Fetch[1] >= 0):
      df.iloc[Fetch[1]][str(ciclo)] = "F"  # marca tabela F
    if ( Decode[1] >= 0):
      df.iloc[Decode[1]][str(ciclo)] = "D" # marca tabela F
    if ( ciclo >= forward ):  # se RAW resolveu, libera stall,registro no proximo ciclo
      stall = False
      if register in writeback_f:
        writeback_f.pop(register)
      if ( Decode[1] >= 0):
        if ( Decode[0][0] == "addf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1

        if ( Decode[0][0] == "mulf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1

        if ( Decode[0][0] == "ld"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1


  if ( DecodeBAR[1] >= 0):
    if ( DecodeBAR[0][0] == "addf"):
      addfBAR[0] = addf[0]
      addf[0] = DecodeBAR
      df.iloc[addf[0][1]][str(ciclo)] = "A0"
    else:
      addfBAR[0] = addf[0]
      addf[0] = [NOP,-1]
    if ( DecodeBAR[0][0] == "mulf"):
      mulfBAR[0] = mulf[0]
      mulf[0] = DecodeBAR
      df.iloc[mulf[0][1]][str(ciclo)] = "M0"
    else:
      mulfBAR[0] = mulf[0]
      mulf[0] = [NOP,-1]
    #adicionar sd/ld e addi
    if ( DecodeBAR[0][0] == "ld" or  DecodeBAR[0][0] == "sd"):
      ldBAR[0] = ld[0]
      ld[0] = DecodeBAR
      df.iloc[ld[0][1]][str(ciclo)] = "L0"
    else:
      ldBAR[0] = ld[0]
      ld[0] = [NOP,-1]
    if ( DecodeBAR[0][0] == "addi"):
      addi[0] = DecodeBAR
      df.iloc[DecodeBAR[0][1]][str(ciclo)] = "I"
  else:
    addfBAR[0] = addf[0]
    addf[0] = [NOP,-1]
    mulfBAR[0] = mulf[0]
    mulf[0] = [NOP,-1]
    #adicionar sd/ld
    ldBAR[0] = ld[0]
    ld[0] = [NOP,-1]
    addi[0] = [NOP,-1]

  for i in range(1,Tempo["addf"]):
    addfBAR[i] = addf[i]
    addf[i] = addfBAR[i-1]
    if ( addf[i][0][0] == "addf"):
      df.iloc[addf[i][1]][str(ciclo)] = "A"+str(i)
    if ( i == Tempo["addf"] - 1) and (addf[i][0][0] == "addf") and (addf[i][0][1] in writeback_f and stall):
        writeback_f.pop(addf[i][0][1])
        #print("descadastra registro ",addf[i][0][1])

  if (addfBAR[Tempo["addf"]-1][1] >= 0):
    df.iloc[addfBAR[Tempo["addf"]-1][1]][str(ciclo)] = "W"
    if (ciclo > last):
      last = ciclo

  addfBAR[Tempo["addf"]-1] = addf[Tempo["addf"]-1]

  for i in range(1,Tempo["mulf"]):
    mulfBAR[i] = mulf[i]
    mulf[i] = mulfBAR[i-1]
    if ( mulf[i][0][0] == "mulf"):
      df.iloc[mulf[i][1]][str(ciclo)] = "M"+str(i)
    if ( i == Tempo["mulf"] - 1) and (mulf[i][0][0] == "mulf") and (mulf[i][0][1] in writeback_f and  stall):
        writeback_f.pop(mulf[i][0][1])
        # print("descadastra registro ",mulf[i][0][1])

  if (mulfBAR[Tempo["mulf"]-1][1] >= 0):
    df.iloc[mulfBAR[Tempo["mulf"]-1][1]][str(ciclo)] = "W"
    if (ciclo > last):
      last = ciclo

  mulfBAR[Tempo["mulf"]-1] = mulf[Tempo["mulf"]-1]
  #adicionar ld/sd
  for i in range(1,Tempo["ld"]):
    ldBAR[i] = ld[i]
    ld[i] = ldBAR[i-1]
    if ( ((ld[i][0][0] == "ld") or (ld[i][0][0] == "sd"))):
      df.iloc[ld[i][1]][str(ciclo)] = "L"+str(i)
    if ( i == Tempo["ld"] - 1) and ((ld[i][0][0] == "ld"))and (ld[i][0][1] in writeback_f and stall): #sd n grava no registrador
        writeback_f.pop(ld[i][0][1])
        #print("descadastra registro ",ld[i][0][1])

  if (ldBAR[Tempo["ld"]-1][1] >= 0):
    if(ldBAR[Tempo["ld"]-1][0][0] == "ld"):
      df.iloc[ldBAR[Tempo["ld"]-1][1]][str(ciclo)] = "W"
    else:
      df.iloc[ldBAR[Tempo["ld"]-1][1]][str(ciclo)] = "-"
    if (ciclo > last):
      last = ciclo

  ldBAR[Tempo["ld"]-1] = ld[Tempo["ld"]-1]

  if writeback_r:
    key, value = next(iter(writeback_r.items()))
    if(ciclo == value):
      writeback_r.pop(key)
      df.iloc[addi[0][1]][str(ciclo)] = "W"
      addi[0] = [NOP,-1]
      if(ciclo > last):
        last = ciclo

  #print(ciclo,writeback_f)
  ciclo+= 1
cpi = last/instrucoes
print("Executou ",instrucoes," instrucoes em ",last," ciclos, com ",n_stall," stalls e CPI=",cpi)
df

Executou  6  instrucoes em  17  ciclos, com  4  stalls e CPI= 2.8333333333333335


,prog,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,"addf f1, f2, f3",F,D,A0,A1,A2,W,,,,,,,,,,,,
1,"mulf f3, f1, f3",,F,D,D,D,M0,M1,M2,M3,M4,W,,,,,,,
2,"addi r2, r2, r1",,,F,F,F,D,I,W,,,,,,,,,,
3,"ld f5, 0(r2)",,,,,,F,D,L0,L1,W,,,,,,,,
4,"addf f4, f3, f5",,,,,,,F,D,D,D,A0,A1,A2,W,,,,
5,"mulf f3, f3, f5",,,,,,,,F,F,F,D,M0,M1,M2,M3,M4,W,


#Parte II - trabalho 01
comentários no código


## Parte II

* Fazer a saída com visualização nas colunas dos estágios e nas células da temporização. Permitir Loop repetido o código.

* O Loop pode ser um parametro interativo como o Maximo_Ciclos.

* Exemplo de como deve ficar a saída:

inst | fetch | dec | exec | wb
---|---      |  ---|   ---|---
addf f1,f2,f3 | 1 | 2 | 3,4,5| 6
addf f1,f1,f3 | 2 | 3,4,5 | 6,7,8| 9
mult f3,f2,f3 | 3,4,5 | 6 | 7,8,9,10,11| 12
addf f6,f4,f5 | 6 | 7 | 8,9,10 | 11


* Deve suportar: addf, multf, ld/sd e addi.





In [ ]:
from IPython.lib.pretty import Breakable
#@title Parte II

import pandas as pd
def software_pipeline(codigo, max_ciclos):
  pc = 0
  max_ciclos = max_ciclos
  last = 0
  instrucoes = len(codigo)
  L = []
  #constroi a tabela
  for i in codigo:
    L.append(inst2text(i))
  tabela = { 'prog': L}


  L = []
  for i in codigo:
    L.append(" ")

  tabela["fetch"] = L
  tabela["decode"] = L
  tabela["exec"] = L
  tabela["write"] = L

  df = pd.DataFrame(data=tabela)
  instrucoes = (len(tabela['prog']))
  NOP = ["nop",0,0,0]
  ciclo = 1
  stall = False
  Fetch = [NOP,-1]
  Decode = [NOP,-1]
  WBack = [NOP,-1]
  addf = []
  addfBAR = []
  for i in range(Tempo["addf"]):
    addf.append([NOP,-1])
    addfBAR.append([NOP,-1])
  mulf = []
  mulfBAR = []
  for i in range(Tempo["mulf"]):
    mulf.append([NOP,-1])
    mulfBAR.append([NOP,-1])
  #acho q n precisa do addi pq é só um ciclo
  #iniciando ld/sd(precisa de um para representar os dois, já que eles seguem o "mesmo" fluxo)
  ld = []
  ldBAR = []
  for i in range(Tempo["ld"]):
    ld.append([NOP,-1])
    ldBAR.append([NOP,-1])

  addi = [[[NOP,-1]]]
  #

  #diferenciar por dois motivos: addi grava em outro registrador e também no caso de conflito no W se o ld for com r1 não necessita ter conflito.
  writeback_f = {}
  writeback_r = {}
  #writeback_f_f => para registradores F(tipo flutuante)
  #writeback_f_s => para registradores R(tipo inteiro)

  forward = 0
  n_stall = 0
  forward2 = 0 #para o conflito no estagio W
  register = 0

  inst = {}
  for i in range(0,instrucoes):
    inst[i] = []
  #Para saída ficar igual foi pedido, fizemos um vetor inst que ira armazenas inst[num_pc] = ('W', ciclo) (o que vai estar fazendo, em que ciclo vai estar fazendo)

  while ciclo < max_ciclos:
    if (not stall):
      FetchBAR = Fetch       # move a instrucao ciclo anterior F -> BARRA
      if (pc < instrucoes):    # Fetch, verifica se ainda tem instrucoes
        Fetch = [codigo[pc],pc]         # Busca nova instrucao e sua linha na tabela
        inst[pc].append(('F', ciclo))  # Marca F na tabela
        pc += 1                         # prepara para buscar a proxima
      else:
        Fetch = [NOP,-1]    # busca NOP

      DecodeBAR = Decode  # move a instrucao ciclo anterior D -> BARRA
      Decode = FetchBAR   # busca a instrucao ciclo anterior F->D
      if ( Decode[1] >= 0):       # senao é NOP marca D na tabela
        inst[Decode[1]].append(('D', ciclo))

      #RESOLVENDO POSSÍVEIS CONFLITOS NO W
      if(not Decode[0][0] == "nop" and not Decode[0][0] == "sd"):
        tempo = ciclo + Tempo[Decode[0][0]] + 1
        # print(Decode, tempo)
        for key in writeback_f.keys():
          if writeback_f[key] == tempo: #se tiver algum registrador F que irá escrever no mesmo ciclo, dá um stall
            stall = True
            forward2 = tempo + 1
            break
      #RESOLVENDO DEPENDÊNCIAS DE DADOS
      if Decode[0][2] in writeback_f or Decode[0][3] in writeback_f:# verifica se tem RAW rs1 e rs2
        if(Decode[0][2] in writeback_f and Decode[0][3] in writeback_f): #resolver conflitos desse tipo => [["mulf",3,1,3],["addi",2,2,1],["ld",5,0,2],["addf",4,3,5]], quando vai testar com o útlimo addf
        #tanto f3 quanto f5 estão no writeback_f, então por ordem ele acaba pegando o f5(segundo if) e dava errado :( #https://excalidraw.com/#json=YBtQq_m-daQlj3r1yVlLG,M6HFuD-r6WQO1yYJA8SWpQ
          if(writeback_f[Decode[0][2]] > writeback_f[Decode[0][3]]):
            forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
            register = Decode[0][2]
          else:
            forward = writeback_f[Decode[0][3]] - 1
            register = Decode[0][3]
          stall = True
        elif ( Decode[0][2] in writeback_f):
          forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
          register = Decode[0][2]            # guarda registro de escrita (para )
          stall = True
        elif ( Decode[0][3] in writeback_f):
          forward = writeback_f[Decode[0][3]] - 1
          register = Decode[0][3]
          stall = True                           # marca STALL

      else:                                    # senao tem dependencia mais, registra onde ira escrever
        if ( Decode[0][0] == "addf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1
        if ( Decode[0][0] == "mulf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1
        #adicionar addi e ld/sd
        if ( Decode[0][0] == "addi"):
          writeback_r[Decode[0][1]] = ciclo+Tempo["addi"]+1
        if ( Decode[0][0] == "ld"): #sd grava na memoria e não em registrador
          writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1

    else: #stall
      n_stall+=1
      DecodeBAR = [NOP,-1]  # Ira propagar NOP para exec
      if ( ciclo == forward2 ):  # se RAW resolveu, libera stall,registro no proximo ciclo
        stall = False
        continue
      if ( Fetch[1] >= 0):
        inst[Fetch[1]].append(('F', ciclo))  # marca tabela F
      if ( Decode[1] >= 0):
        inst[Decode[1]].append(('D', ciclo)) # marca tabela F
      if ( ciclo >= forward ):  # se RAW resolveu, libera stall,registro no proximo ciclo
        stall = False
        if register in writeback_f:
          writeback_f.pop(register)
        if ( Decode[1] >= 0):
          if ( Decode[0][0] == "addf"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1

          if ( Decode[0][0] == "mulf"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1

          if ( Decode[0][0] == "ld"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1


    if ( DecodeBAR[1] >= 0):
      if ( DecodeBAR[0][0] == "addf"):
        addfBAR[0] = addf[0]
        addf[0] = DecodeBAR
        inst[addf[0][1]].append(('E', ciclo))
      else:
        addfBAR[0] = addf[0]
        addf[0] = [NOP,-1]
      if ( DecodeBAR[0][0] == "mulf"):
        mulfBAR[0] = mulf[0]
        mulf[0] = DecodeBAR
        inst[mulf[0][1]].append(('E', ciclo))

      else:
        mulfBAR[0] = mulf[0]
        mulf[0] = [NOP,-1]
      #adicionar sd/ld e addi
      if ( DecodeBAR[0][0] == "ld" or  DecodeBAR[0][0] == "sd"):
        ldBAR[0] = ld[0]
        ld[0] = DecodeBAR
        df.iloc[ld[0][1]][str(ciclo)] = "L0"
        inst[ld[0][1]].append(('E', ciclo))
      else:
        ldBAR[0] = ld[0]
        ld[0] = [NOP,-1]
      if ( DecodeBAR[0][0] == "addi"):
        addi[0] = DecodeBAR
        inst[DecodeBAR[0][1]].append(('E', ciclo))
    else:
      addfBAR[0] = addf[0]
      addf[0] = [NOP,-1]
      mulfBAR[0] = mulf[0]
      mulf[0] = [NOP,-1]
      #adicionar sd/ld
      ldBAR[0] = ld[0]
      ld[0] = [NOP,-1]
      addi[0] = [NOP,-1]

    #AQUI ONDE O "EXEC" VAI ACONTECER
    for i in range(1,Tempo["addf"]):
      addfBAR[i] = addf[i]
      addf[i] = addfBAR[i-1]
      if ( addf[i][0][0] == "addf"):
        inst[addf[i][1]].append(('E', ciclo))
      if ( i == Tempo["addf"] - 1) and (addf[i][0][0] == "addf") and (addf[i][0][1] in writeback_f and stall):
          writeback_f.pop(addf[i][0][1])
          #print("descadastra registro ",addf[i][0][1])

    if (addfBAR[Tempo["addf"]-1][1] >= 0):
      inst[addfBAR[Tempo["addf"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    addfBAR[Tempo["addf"]-1] = addf[Tempo["addf"]-1]

    for i in range(1,Tempo["mulf"]):
      mulfBAR[i] = mulf[i]
      mulf[i] = mulfBAR[i-1]
      if ( mulf[i][0][0] == "mulf"):
        inst[mulf[i][1]].append(('E', ciclo))
      if ( i == Tempo["mulf"] - 1) and (mulf[i][0][0] == "mulf") and (mulf[i][0][1] in writeback_f and  stall):
          writeback_f.pop(mulf[i][0][1])
          # print("descadastra registro ",mulf[i][0][1])

    if (mulfBAR[Tempo["mulf"]-1][1] >= 0):
      inst[mulfBAR[Tempo["mulf"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    mulfBAR[Tempo["mulf"]-1] = mulf[Tempo["mulf"]-1]
    #adicionar ld/sd
    for i in range(1,Tempo["ld"]):
      ldBAR[i] = ld[i]
      ld[i] = ldBAR[i-1]
      if ( ((ld[i][0][0] == "ld") or (ld[i][0][0] == "sd"))):
        inst[ld[i][1]].append(('E', ciclo))
      if ( i == Tempo["ld"] - 1) and ((ld[i][0][0] == "ld"))and (ld[i][0][1] in writeback_f and stall):
          writeback_f.pop(ld[i][0][1])
          #print("descadastra registro ",ld[i][0][1])

    if (ldBAR[Tempo["ld"]-1][1] >= 0):
      if(ldBAR[Tempo["ld"]-1][0][0] == "ld"):
        print('oi')
        inst[ldBAR[Tempo["ld"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    ldBAR[Tempo["ld"]-1] = ld[Tempo["ld"]-1]

    if writeback_r:
      key, value = next(iter(writeback_r.items()))
      if(ciclo == value):
        writeback_r.pop(key)
        inst[addi[0][1]].append(('W', ciclo)) #inst[num_pc] = ('W', ciclo)
        addi[0] = [NOP,-1]
        if(ciclo > last):
          last = ciclo

    #print(ciclo,writeback_f)
    ciclo+= 1
  for key in inst.keys():
    F = []
    D = []
    E = []
    W = []
    for i in inst[key]:

      if(i[0] == 'F'):
        F.append(i[1])
      if(i[0] == 'D'):
        D.append(i[1])
      if(i[0] == 'E'):
        E.append(i[1])
      if(i[0] == 'W'):
        W.append(i[1])
    f = my_str = ','.join([str(elem) for elem in F])
    d = my_str = ','.join([str(elem) for elem in D])
    e = my_str = ','.join([str(elem) for elem in E])
    w = my_str = ','.join([str(elem) for elem in W])
    df.iloc[key]["fetch"] = f
    df.iloc[key]["decode"] = d
    df.iloc[key]["exec"] = e
    df.iloc[key]["write"] = w


  cpi = last/instrucoes
  result = "Executou "+ str(instrucoes) + " instrucoes em " + str(last) + " ciclos, com " + str(n_stall) + " stalls e CPI=" + str(cpi)
  return result,df


## Parte 2 com alterações após entrega do trabalho

In [ ]:
from IPython.lib.pretty import Breakable
#@title Parte II

import pandas as pd
def software_pipeline(codigo, max_ciclos):
  pc = 0
  max_ciclos = max_ciclos
  last = 0
  instrucoes = len(codigo)
  L = []
  #constroi a tabela
  for i in codigo:
    L.append(inst2text(i))
  tabela = { 'prog': L}


  L = []
  for i in codigo:
    L.append(" ")

  tabela["fetch"] = L
  tabela["decode"] = L
  tabela["exec"] = L
  tabela["write"] = L

  df = pd.DataFrame(data=tabela)
  instrucoes = (len(tabela['prog']))
  NOP = ["nop",0,0,0]
  ciclo = 1
  stall = False
  Fetch = [NOP,-1]
  Decode = [NOP,-1]
  WBack = [NOP,-1]
  addf = []
  addfBAR = []
  for i in range(Tempo["addf"]):
    addf.append([NOP,-1])
    addfBAR.append([NOP,-1])
  mulf = []
  mulfBAR = []
  for i in range(Tempo["mulf"]):
    mulf.append([NOP,-1])
    mulfBAR.append([NOP,-1])
  #acho q n precisa do addi pq é só um ciclo
  #iniciando ld/sd(precisa de um para representar os dois, já que eles seguem o "mesmo" fluxo)
  ld = []
  ldBAR = []
  for i in range(Tempo["ld"]):
    ld.append([NOP,-1])
    ldBAR.append([NOP,-1])

  addi = [[[NOP,-1]]]
  #

  #diferenciar por dois motivos: addi grava em outro registrador e também no caso de conflito no W se o ld for com r1 não necessita ter conflito.
  writeback_f = {}
  writeback_r = {}
  #writeback_f_f => para registradores F(tipo flutuante)
  #writeback_f_s => para registradores R(tipo inteiro)

  forward = 0
  n_stall = 0
  forward2 = 0 #para o conflito no estagio W
  register = 0

  inst = {}
  for i in range(0,instrucoes):
    inst[i] = []
  #Para saída ficar igual foi pedido, fizemos um vetor inst que ira armazenas inst[num_pc] = ('W', ciclo) (o que vai estar fazendo, em que ciclo vai estar fazendo)

  while ciclo < max_ciclos:
    if (not stall):
      FetchBAR = Fetch       # move a instrucao ciclo anterior F -> BARRA
      if (pc < instrucoes):    # Fetch, verifica se ainda tem instrucoes
        Fetch = [codigo[pc],pc]         # Busca nova instrucao e sua linha na tabela
        inst[pc].append(('F', ciclo))  # Marca F na tabela
        pc += 1                         # prepara para buscar a proxima
      else:
        Fetch = [NOP,-1]    # busca NOP

      DecodeBAR = Decode  # move a instrucao ciclo anterior D -> BARRA
      Decode = FetchBAR   # busca a instrucao ciclo anterior F->D
      if ( Decode[1] >= 0):       # senao é NOP marca D na tabela
        inst[Decode[1]].append(('D', ciclo))

      #RESOLVENDO POSSÍVEIS CONFLITOS NO W
      if(not Decode[0][0] == "nop" and not Decode[0][0] == "sd"):
        tempo = ciclo + Tempo[Decode[0][0]] + 1
        # print(Decode, tempo)
        for key in writeback_f.keys():
          if writeback_f[key] == tempo: #se tiver algum registrador F que irá escrever no mesmo ciclo, dá um stall
            stall = True
            forward2 = tempo + 1
            break
      #RESOLVENDO DEPENDÊNCIAS DE DADOS
      #alteração pos entrega trabalho
      if Decode[0][0] == 'sd' and Decode[0][1] in writeback_f:
          forward = writeback_f[Decode[0][1]] - 1  # escalona a saida do Stall
          register = Decode[0][1]
          stall = True
      #fim
      if Decode[0][2] in writeback_f or Decode[0][3] in writeback_f:# verifica se tem RAW rs1 e rs2
        if(Decode[0][2] in writeback_f and Decode[0][3] in writeback_f): #resolver conflitos desse tipo => [["mulf",3,1,3],["addi",2,2,1],["ld",5,0,2],["addf",4,3,5]], quando vai testar com o útlimo addf
        #tanto f3 quanto f5 estão no writeback_f, então por ordem ele acaba pegando o f5(segundo if) e dava errado :( #https://excalidraw.com/#json=YBtQq_m-daQlj3r1yVlLG,M6HFuD-r6WQO1yYJA8SWpQ
          if(writeback_f[Decode[0][2]] > writeback_f[Decode[0][3]]):
            forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
            register = Decode[0][2]
          else:
            forward = writeback_f[Decode[0][3]] - 1
            register = Decode[0][3]
          stall = True
        elif ( Decode[0][2] in writeback_f):
          forward = writeback_f[Decode[0][2]] - 1  # escalona a saida do Stall
          register = Decode[0][2]            # guarda registro de escrita (para )
          stall = True
        elif ( Decode[0][3] in writeback_f):
          forward = writeback_f[Decode[0][3]] - 1
          register = Decode[0][3]
          stall = True                           # marca STALL

      else:                                    # senao tem dependencia mais, registra onde ira escrever
        if ( Decode[0][0] == "addf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1
        if ( Decode[0][0] == "mulf"):
          writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1
        #adicionar addi e ld/sd
        if ( Decode[0][0] == "addi"):
          writeback_r[Decode[0][1]] = ciclo+Tempo["addi"]+1
        if ( Decode[0][0] == "ld"): #sd grava na memoria e não em registrador
          writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1

    else: #stall
      n_stall+=1
      DecodeBAR = [NOP,-1]  # Ira propagar NOP para exec
      print(Decode[0][0], ciclo, stall, register, writeback_f)
      if ( ciclo == forward2 ):  # se RAW resolveu, libera stall,registro no proximo ciclo
        stall = False
        continue
      if ( Fetch[1] >= 0):
        inst[Fetch[1]].append(('F', ciclo))  # marca tabela F
      if ( Decode[1] >= 0):
        inst[Decode[1]].append(('D', ciclo)) # marca tabela F
      if ( ciclo >= forward ):  # se RAW resolveu, libera stall,registro no proximo ciclo
        stall = False
        if register in writeback_f:
          writeback_f.pop(register)
        if ( Decode[1] >= 0):
          if ( Decode[0][0] == "addf"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["addf"]+1

          if ( Decode[0][0] == "mulf"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["mulf"]+1

          if ( Decode[0][0] == "ld"):
            writeback_f[Decode[0][1]] = ciclo+Tempo["ld"]+1


    if ( DecodeBAR[1] >= 0):
      if ( DecodeBAR[0][0] == "addf"):
        addfBAR[0] = addf[0]
        addf[0] = DecodeBAR
        inst[addf[0][1]].append(('E', ciclo))
      else:
        addfBAR[0] = addf[0]
        addf[0] = [NOP,-1]
      if ( DecodeBAR[0][0] == "mulf"):
        mulfBAR[0] = mulf[0]
        mulf[0] = DecodeBAR
        inst[mulf[0][1]].append(('E', ciclo))

      else:
        mulfBAR[0] = mulf[0]
        mulf[0] = [NOP,-1]
      #adicionar sd/ld e addi
      if ( DecodeBAR[0][0] == "ld" or  DecodeBAR[0][0] == "sd"):
        ldBAR[0] = ld[0]
        ld[0] = DecodeBAR
        df.iloc[ld[0][1]][str(ciclo)] = "L0"
        inst[ld[0][1]].append(('E', ciclo))
      else:
        ldBAR[0] = ld[0]
        ld[0] = [NOP,-1]
      if ( DecodeBAR[0][0] == "addi"):
        addi[0] = DecodeBAR
        inst[DecodeBAR[0][1]].append(('E', ciclo))
    else:
      addfBAR[0] = addf[0]
      addf[0] = [NOP,-1]
      mulfBAR[0] = mulf[0]
      mulf[0] = [NOP,-1]
      #adicionar sd/ld
      ldBAR[0] = ld[0]
      ld[0] = [NOP,-1]
      addi[0] = [NOP,-1]

    #AQUI ONDE O "EXEC" VAI ACONTECER
    for i in range(1,Tempo["addf"]):
      addfBAR[i] = addf[i]
      addf[i] = addfBAR[i-1]
      if ( addf[i][0][0] == "addf"):
        inst[addf[i][1]].append(('E', ciclo))
      if ( i == Tempo["addf"] - 1) and (addf[i][0][0] == "addf") and (addf[i][0][1] in writeback_f ):
          writeback_f.pop(addf[i][0][1])
          #print("descadastra registro ",addf[i][0][1])

    if (addfBAR[Tempo["addf"]-1][1] >= 0):
      inst[addfBAR[Tempo["addf"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    addfBAR[Tempo["addf"]-1] = addf[Tempo["addf"]-1]

    for i in range(1,Tempo["mulf"]):
      mulfBAR[i] = mulf[i]
      mulf[i] = mulfBAR[i-1]
      if ( mulf[i][0][0] == "mulf"):
        inst[mulf[i][1]].append(('E', ciclo))
      if ( i == Tempo["mulf"] - 1) and (mulf[i][0][0] == "mulf") and (mulf[i][0][1] in writeback_f):
          writeback_f.pop(mulf[i][0][1])
          # print("descadastra registro ",mulf[i][0][1])

    if (mulfBAR[Tempo["mulf"]-1][1] >= 0):
      inst[mulfBAR[Tempo["mulf"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    mulfBAR[Tempo["mulf"]-1] = mulf[Tempo["mulf"]-1]
    #adicionar ld/sd
    for i in range(1,Tempo["ld"]):
      ldBAR[i] = ld[i]
      ld[i] = ldBAR[i-1]
      if ( ((ld[i][0][0] == "ld") or (ld[i][0][0] == "sd"))):
        inst[ld[i][1]].append(('E', ciclo))
      if ( i == Tempo["ld"] - 1) and ((ld[i][0][0] == "ld"))and (ld[i][0][1] in writeback_f):
          writeback_f.pop(ld[i][0][1])
          #print("descadastra registro ",ld[i][0][1])

    if (ldBAR[Tempo["ld"]-1][1] >= 0):
      if(ldBAR[Tempo["ld"]-1][0][0] == "ld"):
        print('oi')
        inst[ldBAR[Tempo["ld"]-1][1]].append(('W', ciclo))
      if (ciclo > last):
        last = ciclo

    ldBAR[Tempo["ld"]-1] = ld[Tempo["ld"]-1]

    if writeback_r:
      key, value = next(iter(writeback_r.items()))
      if(ciclo == value):
        writeback_r.pop(key)
        inst[addi[0][1]].append(('W', ciclo)) #inst[num_pc] = ('W', ciclo)
        addi[0] = [NOP,-1]
        if(ciclo > last):
          last = ciclo

    #print(ciclo,writeback_f)
    ciclo+= 1
  for key in inst.keys():
    F = []
    D = []
    E = []
    W = []
    for i in inst[key]:

      if(i[0] == 'F'):
        F.append(i[1])
      if(i[0] == 'D'):
        D.append(i[1])
      if(i[0] == 'E'):
        E.append(i[1])
      if(i[0] == 'W'):
        W.append(i[1])
    f = my_str = ','.join([str(elem) for elem in F])
    d = my_str = ','.join([str(elem) for elem in D])
    e = my_str = ','.join([str(elem) for elem in E])
    w = my_str = ','.join([str(elem) for elem in W])
    df.iloc[key]["fetch"] = f
    df.iloc[key]["decode"] = d
    df.iloc[key]["exec"] = e
    df.iloc[key]["write"] = w


  cpi = last/instrucoes
  result = "Executou "+ str(instrucoes) + " instrucoes em " + str(last) + " ciclos, com " + str(n_stall) + " stalls e CPI=" + str(cpi)
  return result,df
software_pipeline([["addf",3,5,2],["ld",4,0,1],  ["addf", 4,3,2]], 20)


ld 4 True 0 {3: 6, 4: 6}
addf 6 True 3 {4: 7}
oi


('Executou 3 instrucoes em 10 ciclos, com 2 stalls e CPI=3.3333333333333335',
               prog fetch decode   exec write
 0  addf f3, f5, f2     1      2  3,4,5     6
 1     ld f4, 0(r1)     2    3,4    5,6     7
 2  addf f4, f3, f2   3,4    5,6  7,8,9    10)

## Pontos Extras (até 3 pontos)

Melhor o trabalho com um ou mais dos itens abaixo:

* Adicionar uma interface Gradio
* Ser selecionado o melhor trabalho da turma (banca de professores externos para avaliar)
* Ter uma entrada para digitar a resposta e o simulador faz a correção do exercício



In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [ ]:
from IPython.lib.pretty import Breakable
#@title Interface Gradio

import gradio as gr
import pandas as pd
def run_softwarepipeline(Codigo, max_ciclos):
  try:
    codigo = eval(Codigo)
    return software_pipeline(codigo, max_ciclos)
  except:
    return "Entrada não foi válida, tente novamente, escrevendo como nos exemplos", pd.DataFrame({'prog' : [''], 'fetch': [''], 'decode': [''], 'exec': [''], 'write': ['']})

# run_softwarepipeline('[["addf",3,5,2],["ld",1,0,1]]', 7)
with gr.Blocks() as app:
    def limpar_campos():
        gr.update(code, value="")
        gr.update(num_ciclos, value=1)
        gr.update(output_text = "")
    with gr.Row():
      with gr.Column():
        code = gr.Textbox(label='Código',placeholder="Digite uma lista de códigos...")
        num_ciclos = gr.Slider(minimum= 1, maximum= 50,step=1, label="Escolha o máximo de ciclos")
        with gr.Row():
          clear = gr.Button("Clear")
          submit = gr.Button("Submit")
      with gr.Column():
        output_text = gr.Textbox()
        output = gr.DataFrame(type="pandas", headers=["prog", "fetch", "decode", "exec", "write"], interactive=False)
        reset = gr.Button("Tente novamente", visible=False)

    clear.click(lambda x: gr.update(value=''), [code],[code])
    clear.click(lambda x: gr.update(value=1), [num_ciclos],[num_ciclos])

    reset.click(lambda x: gr.update(value=''), [code],[code])
    reset.click(lambda x: gr.update(value=''), [output_text],[output_text])
    reset.click(lambda x: gr.update(value=1), [num_ciclos],[num_ciclos])
    reset.click(lambda x: gr.update(value=pd.DataFrame({'prog' : [''], 'fetch': [''], 'decode': [''], 'exec': [''], 'write': ['']})), [output],[output])
    reset.click(lambda x: gr.update(visible=False), [reset],[reset])

    submit.click(lambda x: gr.update(visible=True), [reset],[reset])
    submit.click(fn=run_softwarepipeline,
                 inputs=[code, num_ciclos],
                 outputs=[output_text, output])
    examples = gr.Examples(examples=['[["addf",3,5,2],["ld",1,0,1]]', '[["addf",1,2,3],["mulf",3,1,3],["addi",2,2,1],["ld",5,0,2],["addf",4,3,5],["mulf",3,3,5]]'], inputs=[code])


app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://be8a9c4d2a2fb72f85.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
